## Test per progetto AI

E' necessario caricare i file roads_small.json e nodes_small.json

In [75]:
import json
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [76]:
nodes = json.load(open('new_nodes.json', 'r'))
roads = json.load(open('new_roads.json', 'r'))
print("Num nodes", len(nodes))
print("Num roads", len(roads))

Num nodes 2239
Num roads 10138


Create two maps as lookup tables for the nodes

In [77]:
#create a lookup table for point coordinates
node_coords = {point['id']:tuple(point['coordinates']) for point in nodes} #dictonary initialization with comprehension
coords_node = {tuple(point['coordinates']):point['id'] for point in nodes} #tuples are hashable-> can be key to a dictionary

In [78]:
print(node_coords[roads[11]['p1']]) #retrieve coordinates from node ID

(12.55, 45.59)


A questo punto bisogna generare le query degli utenti:

semplificazione: l'orario non influisce sui tempi di percorrenza -> dipendono solo dalla distanza

Consider each trip as starting or ending in one of the nodes: the function below serves to approximate the closest node on the map

Regular transfer (not related to pendolarism) is modeled as a gaussian random variable with respect to location ($\mu = 15km, \sigma = 10km$)  

In [79]:
# compute distance between two nodes

def compute_distance(x_coord_1, y_coord_1, x_coord_2, y_coord_2):
  x_coord_1 = x_coord_1 * 111.32
  x_coord_2 = x_coord_2 * 111.32
  y_coord_1 = 40075 * math.cos(x_coord_1)/360 * y_coord_1;
  y_coord_2 = 40075 * math.cos(x_coord_2)/360 * y_coord_2;
  dist = math.sqrt((y_coord_1 - y_coord_2)**2 + (x_coord_1 - x_coord_2)**2)
  return dist

In [80]:
# use of networkx, creation of graph

import networkx as nx
# use of networkx

G = nx.Graph()

G.add_nodes_from(range(len(nodes)))
for road in roads :
  n1 = int(road.get("p1"))
  n2 = int(road.get("p2"))
  x_coord_1 = float(nodes[n1].get('coordinates')[0])
  y_coord_1 = float(nodes[n1].get('coordinates')[1])
  x_coord_2 = float(nodes[n2].get('coordinates')[0])
  y_coord_2 = float(nodes[n2].get('coordinates')[1])
  dist = compute_distance(x_coord_1, y_coord_1, x_coord_2, y_coord_2)

  G.add_edge(n1, n2, weight=dist)

G.get_edge_data(0, 1)



{'weight': 2698.256500971123}

In [81]:
def generate_ratings(n_driver):
  mean, std = 3.5, 0.5
  x = np.random.normal(mean, std, size=(n_driver, 1))
  ratings = np.empty(shape = (n_driver, 1))
  for i in range(n_driver):
    y = math.ceil(x[i])
    if (y - x[i] > 0.5) :
      ratings[i] = y - 0.5
    else :
      ratings[i] = y
      
  return ratings


In [82]:
# pool of end points 
# define a random number of destinations (e.g. from 8 to 12)
num_dest = random.randrange(8, 12, 1)
pool_dest = []
for i in range(num_dest):
  rand = random.randrange(len(node_coords))
  id_dest = list(node_coords.keys())[rand]
  coord_dest = list(node_coords.values())[rand]
  pool_dest.append({'id':id_dest, 'coordinates':coord_dest})

print(pool_dest)

[{'id': 2136, 'coordinates': (12.39, 45.59)}, {'id': 1588, 'coordinates': (12.18, 45.5)}, {'id': 1477, 'coordinates': (11.75, 45.24)}, {'id': 1166, 'coordinates': (12.05, 45.54)}, {'id': 1470, 'coordinates': (12.37, 45.61)}, {'id': 560, 'coordinates': (12.02, 45.54)}, {'id': 1517, 'coordinates': (12.02, 45.6)}, {'id': 685, 'coordinates': (12.03, 45.38)}]


Minimum path lenght:
- Drivers: 40 km
- Riders: 10 km

In [83]:
def generate_trips(n, gen):
  # needs to define, for each driver/rider, one random starting point, and peak from the pool of destinations a random end points
  trips_start = []
  trips_dest = []
  for i in range(n):
    # start point
    if (gen == 'driver') :
      while True:
        # start point
        rand_s = random.randrange((len(nodes)))
        id_start = list(node_coords.keys())[rand_s]
        # end point
        rand_d = random.randrange(num_dest)
        dest = pool_dest[rand_d]
        dist = nx.shortest_path_length(G, id_start, dest.get('id'), weight='weight')
        if (dist > 40e3) :
          coord_start = list(node_coords.values())[rand_s]
          break
    else :
        while True:
          # start point
          rand_s = random.randrange((len(nodes)))
          id_start = list(node_coords.keys())[rand_s]
          # end point
          rand_d = random.randrange(num_dest)
          dest = pool_dest[rand_d]
          dist = nx.shortest_path_length(G, id_start, dest.get('id'), weight='weight')
          if (dist > 10e3) :
            coord_start = list(node_coords.values())[rand_s]
            break
     
    
    trips_start.append({'id':id_start, 'coordinates':coord_start})
    trips_dest.append(dest)

  return trips_start, trips_dest


In [84]:
def generate_schedules(n):
  #time probability mass: how many requests per 2-hour slot in percentage
  time_prob = {   6 : 0.2,
                  8 : 0.1,
                  10 : 0.07,
                  12 : 0.1,
                  14 : 0.07,
                  16 : 0.15,
                  18 : 0.15,
                  20 : 0.05,
                  22 : 0.05,
                  0 : 0.01,
                  2 : 0.01,
                  4 : 0.04 }

  times = []
  hours = list(time_prob.keys())
  prob = list(time_prob.values())
  for i in range(n):
    times.append(str(random.choices(hours, prob)[0]) + ":00")

  return times

In [85]:
class Driver:
  def __init__(self, id, rating, hour, start_point, end_point):
    self.id = id
    self.rating = rating
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    return "Driver ( id = " + str(self.id) + ", rating = " + str(self.rating) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

class Rider:
  def __init__(self, id, hour, start_point, end_point):
    self.id = id
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    return "Rider ( id = " + str(self.id) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

In [86]:
def generate_data():
  # 1- generate n_drivers and n_riders
  # range: start, end, stepsize
  n_drivers = random.randrange(40, 50, 1)
  n_riders = random.randrange(90, 150, 15)
  print("N_drivers: ", n_drivers)
  print("N_riders: ", n_riders)
  # 2 - generate start point and end point for drivers and riders
  trips_start_drivers, trips_dest_drivers = generate_trips(n_drivers, gen='driver')
  trips_start_riders, trips_dest_riders = generate_trips(n_riders, gen='rider')
  # 3 - generate schedules
  drivers_schedule = generate_schedules(n_drivers)
  riders_schedule = generate_schedules(n_riders)
  # 4- generate ratings for drivers
  ratings = generate_ratings(n_drivers)
  # 5 - parse in a list of objects
  riders = []
  drivers = []
  for i in range(n_riders):
    riders.append( Rider(i, riders_schedule[i], trips_start_riders[i], trips_dest_riders[i]))

  for i in range(n_drivers):
    drivers.append( Driver(i, ratings[i][0], drivers_schedule[i], trips_start_drivers[i], trips_dest_drivers[i]))

  return drivers, riders



In [87]:
drivers, riders = generate_data()

for i in range(5) :
  print(drivers[i])

print("\n\n***********************************************************************\n\n")

for i in range(5) :
  print(riders[i])


N_drivers:  43
N_riders:  90
Driver ( id = 0, rating = 4.0, hour = 6:00, start = {'id': 1484, 'coordinates': (12.42, 45.64)}, dest = {'id': 685, 'coordinates': (12.03, 45.38)})
Driver ( id = 1, rating = 2.5, hour = 8:00, start = {'id': 2112, 'coordinates': (12.62, 45.63)}, dest = {'id': 685, 'coordinates': (12.03, 45.38)})
Driver ( id = 2, rating = 3.5, hour = 8:00, start = {'id': 1623, 'coordinates': (12.5, 45.58)}, dest = {'id': 1477, 'coordinates': (11.75, 45.24)})
Driver ( id = 3, rating = 5.0, hour = 6:00, start = {'id': 2230, 'coordinates': (12.71, 45.6)}, dest = {'id': 1517, 'coordinates': (12.02, 45.6)})
Driver ( id = 4, rating = 4.0, hour = 14:00, start = {'id': 358, 'coordinates': (11.74, 45.23)}, dest = {'id': 1470, 'coordinates': (12.37, 45.61)})


***********************************************************************


Rider ( id = 0, hour = 22:00, start = {'id': 289, 'coordinates': (12.21, 45.62)}, dest = {'id': 560, 'coordinates': (12.02, 45.54)})
Rider ( id = 1, hour =

In [99]:
from datetime import datetime
from datetime import timedelta

# use of Dijkstra for shortest path

drivers_path = []
riders_path = []

# 50 km/h
avg_vel = 50
date_format_str = '%H:%M'

for driver in drivers:
  spath = nx.dijkstra_path(G, driver.start_point.get('id'), driver.end_point.get('id'))
  dpath = [{'node' : spath[0], 'hour': driver.hour}]
  dist = 0


  for i in range(1, len(spath)):
    # retrieve distance and convert into km
    
    # distance in km
    dist += (G.get_edge_data(spath[i-1], spath[i]).get('weight')) / 1e3
    time = dist / avg_vel
    time_1 = datetime.strptime(driver.hour, date_format_str)
    time_2 = time_1 + timedelta(hours=time)
    str_time = time_2.strftime(date_format_str)

    dpath.append({'node': spath[i], 'hour':str_time})



  drivers_path.append({'id_driver': driver.id, 'shortest_path' : dpath})
  total_dist = nx.shortest_path_length(G, driver.start_point.get('id'), driver.end_point.get('id'), weight='weight')
  #print("Driver #" + str(driver.id) + ": start at " + driver.hour + " , end at " + dpath[len(dpath)-1].get('hour'))
  #print("    -> total dist km: ", total_dist/1e3)


for rider in riders:
  spath = nx.dijkstra_path(G, rider.start_point.get('id'), rider.end_point.get('id'))
  riders_path.append({'id_rider': rider.id, 'shortest_path' : spath})

print(drivers_path[0])
#print(riders_path)

{'id_driver': 0, 'shortest_path': [{'node': 1484, 'hour': '6:00'}, {'node': 427, 'hour': '06:01'}, {'node': 1482, 'hour': '06:01'}, {'node': 173, 'hour': '06:01'}, {'node': 164, 'hour': '06:07'}, {'node': 162, 'hour': '06:12'}, {'node': 161, 'hour': '06:20'}, {'node': 314, 'hour': '06:25'}, {'node': 309, 'hour': '06:31'}, {'node': 510, 'hour': '06:31'}, {'node': 523, 'hour': '06:38'}, {'node': 312, 'hour': '06:38'}, {'node': 508, 'hour': '06:38'}, {'node': 514, 'hour': '06:42'}, {'node': 783, 'hour': '06:49'}, {'node': 889, 'hour': '06:54'}, {'node': 107, 'hour': '06:54'}, {'node': 888, 'hour': '07:00'}, {'node': 787, 'hour': '07:03'}, {'node': 788, 'hour': '07:04'}, {'node': 276, 'hour': '07:11'}, {'node': 1152, 'hour': '07:11'}, {'node': 397, 'hour': '07:11'}, {'node': 1175, 'hour': '07:11'}, {'node': 1882, 'hour': '07:11'}, {'node': 1831, 'hour': '07:11'}, {'node': 1528, 'hour': '07:18'}, {'node': 1524, 'hour': '07:20'}, {'node': 1097, 'hour': '07:20'}, {'node': 756, 'hour': '07:21'

In [97]:
# find matching, since we assumed same dest

drivers_match = [{} for _ in range(len(drivers))]

# couple drivers and riders with same dest

for driver in drivers :
  drivers_match[driver.id] = {'id_driver' : driver.id, 'list_of_riders' : []}
  for rider in riders :
    if driver.end_point.get('id') == rider.end_point.get('id') :
      drivers_match[driver.id].get('list_of_riders').append(rider.id)

print(drivers_match)


[{'id_driver': 0, 'list_of_riders': [7, 15, 27, 33, 40, 51, 69, 78, 79, 80, 81]}, {'id_driver': 1, 'list_of_riders': [7, 15, 27, 33, 40, 51, 69, 78, 79, 80, 81]}, {'id_driver': 2, 'list_of_riders': [16, 28, 36, 39, 49, 50, 58, 60, 63, 75, 89]}, {'id_driver': 3, 'list_of_riders': [2, 5, 30, 35, 47, 53, 73, 82]}, {'id_driver': 4, 'list_of_riders': [1, 12, 13, 18, 21, 37, 41, 44, 48, 52, 57, 68, 84, 86, 88]}, {'id_driver': 5, 'list_of_riders': [7, 15, 27, 33, 40, 51, 69, 78, 79, 80, 81]}, {'id_driver': 6, 'list_of_riders': [6, 8, 11, 19, 20, 24, 32, 54, 56, 64, 65, 70, 83, 85]}, {'id_driver': 7, 'list_of_riders': [1, 12, 13, 18, 21, 37, 41, 44, 48, 52, 57, 68, 84, 86, 88]}, {'id_driver': 8, 'list_of_riders': [1, 12, 13, 18, 21, 37, 41, 44, 48, 52, 57, 68, 84, 86, 88]}, {'id_driver': 9, 'list_of_riders': [7, 15, 27, 33, 40, 51, 69, 78, 79, 80, 81]}, {'id_driver': 10, 'list_of_riders': [1, 12, 13, 18, 21, 37, 41, 44, 48, 52, 57, 68, 84, 86, 88]}, {'id_driver': 11, 'list_of_riders': [0, 3, 4

In [ ]:
# create file input.dat (obsolete (?))
f = open("input.dat", "w")
content = "DIMENSION : " + str(len(nodes)) + "\n"
content += "DRIVERS : " + str(len(drivers)) + "\n"
content += "RIDERS : " + str(len(riders)) + "\n"
content += "NODE_COORD_SECTION\n"
for node in nodes :
  content += str(node.get("id")) + " " + str(node.get("coordinates")[0]) + " " + str(node.get("coordinates")[1]) + "\n"
content += "ROADS_SECTION\n"
for road in roads :
  content += str(road.get("p1")) + " " + str(road.get("p2")) + "\n"
content += "DRIVERS_STARTING_COORD\n"
for driver in drivers:
  content += str(driver.start_point.get("id")) + " " + str(driver.start_point.get("coordinates")[0]) + " " + str(driver.start_point.get("coordinates")[1]) + "\n"
content += "DRIVERS_DESTINATION_COORD\n"
for driver in drivers:
  content += str(driver.end_point.get("id")) + " " + str(driver.end_point.get("coordinates")[0]) + " " + str(driver.end_point.get("coordinates")[1]) + "\n"
content += "RIDERS_STARTING_COORD\n"
for rider in riders:
  content += str(rider.start_point.get("id")) + " " + str(rider.start_point.get("coordinates")[0]) + " " + str(rider.start_point.get("coordinates")[1]) + "\n"
content += "RIDERS_DESTINATION_COORD\n"
for rider in riders:
  content += str(rider.end_point.get("id")) + " " + str(rider.end_point.get("coordinates")[0]) + " " + str(rider.end_point.get("coordinates")[1]) + "\n"
content += "EOF"
f.write(content)
f.close()